In [1]:

#Loading dataset
import numpy as np 
import pandas as pd 


import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))



/kaggle/input/customer-churn/Churn-Data.csv


In [2]:
ds = pd.read_csv("/kaggle/input/customer-churn/Churn-Data.csv", index_col=0)

In [3]:
# first things first
#understanding data
print(ds.shape)
ds.head()

(5634, 20)


,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,TV_Streaming,Movie_Streaming,Contract,PaperlessBilling,Method_Payment,Charges_Month,TotalCharges,Churn
cID,,,,,,,,,,,,,,,,,,,,
4223-BKEOR,Female,0,No,Yes,21,Yes,No,DSL,Yes,No,Yes,No,No,Yes,One year,No,Mailed check,64.85,1336.8,No
6035-RIIOM,Female,0,No,No,54,Yes,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,Yes,Bank transfer (automatic),97.20,5129.45,No
3797-VTIDR,Male,0,Yes,No,1,No,No phone service,DSL,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,23.45,23.45,Yes
2568-BRGYX,Male,0,No,No,4,Yes,No,Fiber optic,No,No,No,No,No,No,Month-to-month,Yes,Electronic check,70.20,237.95,Yes
2775-SEFEE,Male,0,No,Yes,0,Yes,Yes,DSL,Yes,Yes,No,Yes,No,No,Two year,Yes,Bank transfer (automatic),61.90,,No


In [4]:
ds.describe()

,SeniorCitizen,tenure,Charges_Month
count,5634.000000,5634.000000,5634.000000
mean,0.160809,32.373092,64.864253
std,0.367388,24.424539,30.089324
min,0.000000,0.000000,18.250000
25%,0.000000,9.000000,35.750000
50%,0.000000,29.000000,70.525000
75%,0.000000,55.000000,89.937500
max,1.000000,72.000000,118.600000


In [5]:
#Let us understand data types and make changes if they are not correct
ds.dtypes

gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
TV_Streaming         object
Movie_Streaming      object
Contract             object
PaperlessBilling     object
Method_Payment       object
Charges_Month       float64
TotalCharges         object
Churn                object
dtype: object

In [6]:
#updating data types
ds['TotalCharges'] = pd.to_numeric(ds['TotalCharges'], errors='coerce')
ds['SeniorCitizen']=ds['SeniorCitizen'].astype(int)
# Checking data types after conversion
print(ds.dtypes)


gender               object
SeniorCitizen         int64
Partner              object
Dependents           object
tenure                int64
PhoneService         object
MultipleLines        object
InternetService      object
OnlineSecurity       object
OnlineBackup         object
DeviceProtection     object
TechSupport          object
TV_Streaming         object
Movie_Streaming      object
Contract             object
PaperlessBilling     object
Method_Payment       object
Charges_Month       float64
TotalCharges        float64
Churn                object
dtype: object


In [7]:
#seperating catergorical and numerical variables
# categorical Variables
categorical_variables = [col for col in ds.columns if
                        ds[col].dtype == "object"]

print(categorical_variables)

# categorical Variables
numeric_variables = [col for col in ds.columns if
                        ds[col].dtype != "object"]

print(numeric_variables)

['gender', 'Partner', 'Dependents', 'PhoneService', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection', 'TechSupport', 'TV_Streaming', 'Movie_Streaming', 'Contract', 'PaperlessBilling', 'Method_Payment', 'Churn']
['SeniorCitizen', 'tenure', 'Charges_Month', 'TotalCharges']


In [8]:
#checking null values
ds.isnull().sum()


gender               0
SeniorCitizen        0
Partner              0
Dependents           0
tenure               0
PhoneService         0
MultipleLines        0
InternetService      0
OnlineSecurity       0
OnlineBackup         0
DeviceProtection     0
TechSupport          0
TV_Streaming         0
Movie_Streaming      0
Contract             0
PaperlessBilling     0
Method_Payment       0
Charges_Month        0
TotalCharges        10
Churn                0
dtype: int64

In [9]:
# as null values are few , I decided to drop them. There are various ways to handle missing values.
ds_new=ds.dropna(subset=['TotalCharges'])

In [10]:
#checking null values again
ds_new.isnull().sum()


gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
TV_Streaming        0
Movie_Streaming     0
Contract            0
PaperlessBilling    0
Method_Payment      0
Charges_Month       0
TotalCharges        0
Churn               0
dtype: int64

In [11]:
# checking outliers with help of skewness and kurtosis 
print(ds['tenure'].skew())
print(ds['tenure'].kurt())
print(ds['Charges_Month'].skew())
print(ds['Charges_Month'].kurt())

0.23994628255612013
-1.3780275276506855
-0.22624795699600828
-1.2605568333807544


In [12]:
# To determine the threshold value for outliers
def outlier_thresholds(dataframe, variable, low_quantile=0.05, up_quantile=0.95):
    quantile_one = dataframe[variable].quantile(low_quantile)
    quantile_three = dataframe[variable].quantile(up_quantile)
    interquantile_range = quantile_three - quantile_one
    up_limit = quantile_three + 1.5 * interquantile_range
    low_limit = quantile_one - 1.5 * interquantile_range
    return low_limit, up_limit

In [13]:
# Are there any outliers in the variables
def has_outliers(dataframe, numeric_columns, plot=False):
   # variable_names = []
    for col in numeric_columns:
        low_limit, up_limit = outlier_thresholds(dataframe, col)
        if dataframe[(dataframe[col] > up_limit) | (dataframe[col] < low_limit)].any(axis=None):
            number_of_outliers = dataframe[(dataframe[col] > up_limit) | (dataframe[col] < low_limit)].shape[0]
            print(col, " : ", number_of_outliers, "outliers")
            #variable_names.append(col)
            if plot:
                sns.boxplot(x=dataframe[col])
                plt.show()
    #return variable_names

In [14]:
for var in numeric_variables:
    print(var, "has " , has_outliers(ds_new, [var]),  "Outliers")

SeniorCitizen has  None Outliers
tenure has  None Outliers
Charges_Month has  None Outliers
TotalCharges has  None Outliers


In [15]:
#I converted columns which has "yes" , "no" values to "0" and "1"


def convert_yes_no_to_numeric(df):
    # Loop through each column in the DataFrame
    for column in df.columns:
        # Check if the column contains only "Yes" and "No" values
        if df[column].dtype == 'object' and set(df[column].unique()) <= {"Yes", "No"}:
            df[column] = df[column].map({"Yes": 1, "No": 0})
    return df

In [16]:
convert_yes_no_to_numeric(ds_new)

/tmp/ipykernel_18/40802915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].map({"Yes": 1, "No": 0})
/tmp/ipykernel_18/40802915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[column] = df[column].map({"Yes": 1, "No": 0})
/tmp/ipykernel_18/40802915.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-doc

,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,OnlineBackup,DeviceProtection,TechSupport,TV_Streaming,Movie_Streaming,Contract,PaperlessBilling,Method_Payment,Charges_Month,TotalCharges,Churn
cID,,,,,,,,,,,,,,,,,,,,
4223-BKEOR,Female,0,0,1,21,1,No,DSL,Yes,No,Yes,No,No,Yes,One year,0,Mailed check,64.85,1336.80,0
6035-RIIOM,Female,0,0,0,54,1,Yes,Fiber optic,No,Yes,No,No,Yes,Yes,Two year,1,Bank transfer (automatic),97.20,5129.45,0
3797-VTIDR,Male,0,1,0,1,0,No phone service,DSL,No,No,No,No,No,No,Month-to-month,1,Electronic check,23.45,23.45,1
2568-BRGYX,Male,0,0,0,4,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,70.20,237.95,1
4291-SHSBH,Male,0,0,0,7,1,No,Fiber optic,No,No,No,No,No,No,Month-to-month,1,Electronic check,69.55,521.35,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0684-AOSIH,Male,0,1,0,1,1,No,Fiber optic,Yes,No,No,No,Yes,Yes,Month-to-month,1,Electronic check,95.00,95.00,1
5982-PSMKW,Female,0,1,1,23,1,Yes,DSL,Yes,Yes,Yes,Yes,Yes,Yes,Two year,1,Credit card (automatic),91.10,2198.30,0
8044-BGWPI,Male,0,1,1,12,1,No,No,No internet service,No internet service,No internet service,No internet service,No internet service,No internet service,Month-to-month,1,Electronic check,21.15,306.05,0


In [17]:
#for all other columns , I applied hot vector encoding

def one_hot_encode_columns(df, columns):
    # Apply one-hot encoding to the specified columns
    df = pd.get_dummies(df, columns=columns, prefix='', prefix_sep='')
    
    # Optionally, reorder columns if needed (here it keeps the same order except encoded columns are moved to the end)
    original_columns = [col for col in df.columns if col not in columns]
    df = df[original_columns + [col for col in df.columns if col not in original_columns]]
    
    return df





# Specify columns to apply one-hot encoding
columns_to_encode = ['gender', 'MultipleLines', 'InternetService', 'OnlineSecurity', 'OnlineBackup', 'DeviceProtection','TechSupport','TV_Streaming','Movie_Streaming','Contract','Method_Payment']

# Apply one-hot encoding
df_encoded = one_hot_encode_columns(ds_new, columns_to_encode)
df_encoded=df_encoded.astype(int)
print(df_encoded)

            SeniorCitizen  Partner  Dependents  tenure  PhoneService  \
cID                                                                    
4223-BKEOR              0        0           1      21             1   
6035-RIIOM              0        0           0      54             1   
3797-VTIDR              0        1           0       1             0   
2568-BRGYX              0        0           0       4             1   
4291-SHSBH              0        0           0       7             1   
...                   ...      ...         ...     ...           ...   
0684-AOSIH              0        1           0       1             1   
5982-PSMKW              0        1           1      23             1   
8044-BGWPI              0        1           1      12             1   
7450-NWRTR              1        0           0      12             1   
4795-UXVCJ              0        0           0      26             1   

            PaperlessBilling  Charges_Month  TotalCharges  Chur

In [18]:
#Its time to apply models
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_predict
from sklearn.metrics import f1_score, precision_score
from sklearn.linear_model import LogisticRegression  
from sklearn.neighbors import KNeighborsClassifier  
from sklearn.svm import SVC
from catboost import CatBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier 
from sklearn.ensemble import RandomForestClassifier

from sklearn.model_selection import train_test_split
from sklearn import preprocessing
from sklearn.metrics import accuracy_score,recall_score
from xgboost import XGBClassifier

from sklearn.model_selection import cross_val_score, GridSearchCV
X = df_encoded.drop("Churn",axis=1)
y = df_encoded["Churn"]
# Train-Test Separation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.20)
scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.transform(X_test)




   

In [19]:
models = {
    'Logistic Regression': LogisticRegression(max_iter=1000, solver='lbfgs'),
    'Random Forest': RandomForestClassifier(),
    'Support Vector Machine': SVC(),
    'K-Nearest Neighbors': KNeighborsClassifier(),
    'Decision Tree': DecisionTreeClassifier(),
    'Gradient Boosting': GradientBoostingClassifier(),
    
}
def evaluate_models(models, X_train, y_train, X_test, y_test):
    results = []
    for name, model in models.items():
        # Train the model
        model.fit(X_train, y_train)
        # Make predictions
        y_pred = model.predict(X_test)
        # Calculate accuracy and F1 score
        accuracy = accuracy_score(y_test, y_pred)
        f1 = f1_score(y_test, y_pred)
        # Append results
        results.append({
            'Model': name,
            'Accuracy': accuracy,
            'F1 Score': f1
        })
    return results

# Evaluate the models
results = evaluate_models(models, X_train_scaled, y_train, X_test_scaled, y_test)

# Convert results to DataFrame and print
results_df = pd.DataFrame(results)
print(results_df)

                    Model  Accuracy  F1 Score
0     Logistic Regression  0.797333  0.577778
1           Random Forest  0.774222  0.503906
2  Support Vector Machine  0.793778  0.536000
3     K-Nearest Neighbors  0.750222  0.519658
4           Decision Tree  0.715556  0.455782
5       Gradient Boosting  0.804444  0.565217


In [20]:
# tuning the model
from sklearn.model_selection import GridSearchCV

# Define the parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [50, 100, 200],
    'max_depth': [None, 10, 20, 30],
    'min_samples_split': [2, 5, 10]
}

# Instantiate the model
rf = RandomForestClassifier()

# Perform grid search
grid_search_rf = GridSearchCV(estimator=rf, param_grid=param_grid_rf, cv=5, scoring='f1')
grid_search_rf.fit(X_train_scaled, y_train)

# Print the best parameters and best score
print("Best parameters for Random Forest:", grid_search_rf.best_params_)
print("Best F1 score for Random Forest:", grid_search_rf.best_score_)

Best parameters for Random Forest: {'max_depth': 10, 'min_samples_split': 5, 'n_estimators': 100}
Best F1 score for Random Forest: 0.5613456794197511


In [21]:
updated_model = RandomForestClassifier(max_depth=10, min_samples_split= 2,n_estimators= 100)
updated_model.fit(X_train_scaled, y_train)
        # Make predictions
y_pred = updated_model.predict(X_test_scaled)
        # Calculate accuracy and F1 score
accuracy = accuracy_score(y_test, y_pred)
f1 = f1_score(y_test, y_pred)
print(accuracy,f1)

0.7928888888888889 0.5404339250493098


In [22]:
from sklearn.ensemble import VotingClassifier

# Create a voting classifier
voting_clf = VotingClassifier(estimators=[
    ('lr', LogisticRegression(max_iter=1000)),
    ('rf', RandomForestClassifier(**grid_search_rf.best_params_)),
    ('svc', SVC(probability=True)),
    ('gb', GradientBoostingClassifier())
], voting='soft')

# Train the voting classifier
voting_clf.fit(X_train_scaled, y_train)

# Evaluate the voting classifier
y_pred_voting = voting_clf.predict(X_test_scaled)
accuracy_voting = accuracy_score(y_test, y_pred_voting)
f1_voting = f1_score(y_test, y_pred_voting)

print("Voting Classifier Accuracy:", accuracy_voting)
print("Voting Classifier F1 Score:", f1_voting)

Voting Classifier Accuracy: 0.8044444444444444
Voting Classifier F1 Score: 0.5564516129032259


In [23]:
import joblib

# Save the best model
joblib.dump(grid_search_rf.best_estimator_, 'best_random_forest_model.pkl')


['best_random_forest_model.pkl']